### Time-Weighted Vector Retriever

In [ ]:
"""
< Time-Weighted Vector Retriever >

가장 최근에 이용된 문서를 기준으로 먼저 참고하도록 하여 답변의 Freshness를 유지할 수 있음
(개념 : 시간이 지난 만큼 페널티를 준다...!)

Scoring 방법 = semantic_similarity + (1.0 - decay_rate)^hours_passed

**pip install faiss-gpu / faiss-cpu 필요
"""

In [4]:
# Libraries
from datetime import datetime, timedelta

import faiss
from langchain.docstore import InMemoryDocstore
from langchain.retrievers import TimeWeightedVectorStoreRetriever
from langchain.schema import Document
from langchain_community.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# Embedding --> Vector DB
model_name = "jhgan/ko-sbert-nli" # 한국말 임베딩 모델 
encode_kwargs = {"normalize_embeddings":True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs
)

# Initialize the vectorstore as empty
embedding_size=768 # 임베딩 싸이즈 설정
index = faiss.IndexFlatL2(embedding_size)
vectorstore = FAISS(ko_embedding, index, InMemoryDocstore({}), {})
retriever = TimeWeightedVectorStoreRetriever(
    vectorstore=vectorstore,
    decay_rate=0.99, # 0~1: 1에 가까울수록 시간에 가까운 유사 정보로 가져옴
    k=1
)
# decay_rate는 0.5정도로 시작하여 결과를 보고 조절하는게 적합함.

In [6]:
# 시간 데이터 생성 및 추가
yesterday = datetime.now() - timedelta(days=1)

# Document 데이터 추가
retriever.add_documents(
    [
        Document(
            page_content="영어는 훌륭합니다.",
            metadata={"last_accessed_at":yesterday} # 어제 데이터
        )
    ]
)

retriever.add_documents([Document(page_content="한국어는 훌륭합니다")]) # 오늘 데이터

['4a2808d0-d08c-4793-b5b5-de48e19ec188']

In [3]:
## decay_rate = 0.01
# 시간 가중치를 줄였기 때문에, 지나간 시간을 거이 고려하지않고, 유사한 문서를 가져옴.
retriever.get_relevant_documents("영어가 좋아요")

[Document(page_content='영어는 훌륭합니다.', metadata={'last_accessed_at': datetime.datetime(2024, 6, 12, 17, 54, 23, 745626), 'created_at': datetime.datetime(2024, 6, 12, 17, 54, 1, 645939), 'buffer_idx': 0})]

In [7]:
## decay_rate = 0.99
# 시간 가중치를 극단적으로 올렸기 때문에, 지나간 시간을 무조건 고려하여, 유사한 문서를 가져옴.
# (유사 하던지 말던지,,, 시간만 따져서 가장 최근 문서를 가져오게 된것...)
retriever.get_relevant_documents("영어가 좋아요")

[Document(page_content='한국어는 훌륭합니다', metadata={'last_accessed_at': datetime.datetime(2024, 6, 12, 17, 57, 19, 814103), 'created_at': datetime.datetime(2024, 6, 12, 17, 57, 17, 677818), 'buffer_idx': 1})]